# 欠落変数バイアス

線形回帰モデルに含めるべき説明変数を含めず、外生性の仮定が満たされない場合に生じるバイアスのことを **欠落変数バイアス（omitted variable bias）** という。


## 背景

### 外生性

単回帰モデル$Y = \alpha + \beta X + u$を例にとる。

:::{admonition} 外生性

説明変数$X$と誤差項$u$が

$$
E(u | X) = 0
$$

を満たすとき、$X$は外生変数であるという。

また、$X$が外生変数であるならば、

$$
E(u) = 0, E(X, u) = 0
$$

と、説明変数$X$と誤差項$u$の無相関

$$
\operatorname{Cov}(X, u) = 0
$$

が成り立つ

:::

:::{dropdown} 証明
$$
E(X u) = E_{X}[ E(X u | X) ] = E_{X}[ X \underbrace{ E(u|X) }_{ =0 } ] = E_{X}(X\cdot 0) = 0
\\
E(u) = E_{X}[ \underbrace{ E(u|X) }_{ =0 } ] = 0
$$


$$
\operatorname{Cov}(X, u)
= \underbrace{ E(X u) }_{ =0 } - E(X) \underbrace{ E(u) }_{ =0 }
= 0
$$

:::


### OLS推定量の一致性

単回帰モデル $Y=\alpha+\beta X+u$ は両辺を$X$と共分散をとると

$$
\begin{aligned}
\operatorname{Cov}(X, Y)
&= \operatorname{Cov}(X, \alpha + \beta X + u)\\
&= \underbrace{ \operatorname{Cov}(X, \alpha) }_{定数との共分散 → 0}
+ \beta  \underbrace{ \operatorname{Cov}(X, X) }_{=\operatorname{Var}(X) }
+\operatorname{Cov}(X, u) \\
&= \beta \operatorname{Var}(X) + \operatorname{Cov}(X, u)\\
\end{aligned}
$$

傾き係数のOLS推定量に代入すると

$$
\begin{aligned}
\hat{\beta} &= \frac{\operatorname{Cov}(X, Y)}{\operatorname{Var}(X)}\\
&= \frac{ \beta \operatorname{Var}(X) + \operatorname{Cov}(X, u) }{\operatorname{Var}(X)}\\
&= \beta + \frac{\operatorname{Cov}(X, u)}{\operatorname{Var}(X)}
\end{aligned}
$$

という形に整理できる。


$X$が外生変数のとき、$\operatorname{Cov}(X, u) = 0$より


$$
\hat{\beta} = \beta + \frac{ \overbrace{\operatorname{Cov}(X, u)}^{=0}  }{\operatorname{Var}(X)}
= \beta
$$

となり、OLS推定量$\hat{\beta}$と真のパラメータ$\beta$は一致する。


## 欠落変数バイアス

単回帰モデル

$$
Y=\alpha+\beta X+u
$$

を構築することを考える。

説明変数は$X$のみであるため、それ以外の$Y$の変動は$u$に含まれることになる。

もし、真のデータ生成過程において、変数$Z$があり真の誤差が$v$で

$$
Y = \alpha + \beta X + \theta Z + v
$$

のようになっていたとすると、単回帰モデルの誤差項$u$には$\theta Z$が含まれていることになる。

$$
Y = \alpha + \beta X + \underbrace{ u }_{\theta Z + v}
$$

もし$Z$が$Y$に影響を与えているだけでなく、$X$にも影響を与えている（=$Z$は交絡因子）であるとすると、$X$は外生性を満たさない

$$
\operatorname{Cov}(X, u) \neq 0
$$

ということになる。そうなるとOLS推定量

$$
\hat{\beta} = \beta + \frac{\operatorname{Cov}(X, u)}{\operatorname{Var}(X)}
$$

は

$$
\frac{\operatorname{Cov}(X, u)}{\operatorname{Var}(X)}
$$

の項が消えないため、その分だけ真のパラメータ$\beta$からずれることになる


## 数値例

人工データを生成し、欠落変数バイアスが $\frac{\operatorname{Cov}(X, u)}{\operatorname{Var}(X)}$ であることを確認する

In [13]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

# Data Generation
def simulate(n=20000, alpha=1.0, beta=2.0, theta=3.0, delta=0.8, seed=42):
    rng = np.random.default_rng(seed)
    Z = rng.normal(size=n)          # confounder
    eps = rng.normal(size=n)        # shock to X
    v = rng.normal(size=n)          # shock to Y
    X = delta * Z + eps             # X depends on Z if delta≠0
    Y = alpha + beta * X + theta * Z + v
    u = theta * Z + v              # omitted-term when regressing Y on X only
    return pd.DataFrame({"Y": Y, "X": X, "Z": Z, "u": u})

beta_true = 2.0
df = simulate(delta=0.8, theta=3.0, beta=beta_true, n=1000, seed=123)

omit_model = smf.ols(formula='Y ~ X', data=df).fit()
full_model = smf.ols(formula='Y ~ X + Z', data=df).fit()
beta_hat_omit = omit_model.params["X"]
beta_hat_full = full_model.params["X"]

# Empirical bias
cov_Xu = np.cov(df["X"], df["u"], bias=True)[0,1]
var_X  = np.var(df["X"])
bias_empirical = cov_Xu / var_X

pd.DataFrame({
    "quantity": [
        "β (true)",
        "β_hat_omit (Y~X)",
        "β_hat_full (Y~X+Z)",
        "Bias (β_hat_omit − β)",
        "Cov(X,u) / Var(X)"
    ],
    "value": [
        beta_true,
        beta_hat_omit,
        beta_hat_full,
        beta_hat_omit - beta_true,
        bias_empirical
    ]
}).round(3)

,quantity,value
0,β (true),2.000
1,β_hat_omit (Y~X),3.394
2,β_hat_full (Y~X+Z),1.964
3,Bias (β_hat_omit − β),1.394
4,"Cov(X,u) / Var(X)",1.394


## 参考文献

- [25. 重回帰分析（脱落変数バイアス・処置後変数バイアス）](http://www.ner.takushoku-u.ac.jp/masano/class_material/waseda/keiryo/R25_reg8_OBV.html)
- 安井翔太. (2020). 効果検証入門: 正しい比較のための因果推論 計量経済学の基礎. Gijutsu hyōronsha.
- 星野匡郎, 田中久稔, & 北川梨津. (2023). R による実証分析: 回帰分析から因果分析へ. 株式会社 オーム社.


### 関連論文（積読リスト）
- [[2302.06518] SelectionBias: An R Package for Bounding Selection Bias](https://arxiv.org/abs/2302.06518)
  - RでSV boundsというバイアスの幅を推定するパッケージ
- [[2208.00552] The Effect of Omitted Variables on the Sign of Regression Coefficients](https://arxiv.org/abs/2208.00552)